In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('Hitters.csv')
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
4,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N


In [3]:
df.shape

(322, 20)

In [4]:
df.isna().sum()

AtBat         0
Hits          0
HmRun         0
Runs          0
RBI           0
Walks         0
Years         0
CAtBat        0
CHits         0
CHmRun        0
CRuns         0
CRBI          0
CWalks        0
League        0
Division      0
PutOuts       0
Assists       0
Errors        0
Salary       59
NewLeague     0
dtype: int64

In [5]:
df['Salary']=df['Salary'].fillna(df['Salary'].mean())

In [6]:
categorical_columns = df.select_dtypes(exclude=[np.number]).columns.tolist()

In [7]:
for cols in categorical_columns:
    dummies=pd.get_dummies(df[cols],drop_first=True)
    dummies=dummies.astype(int)
    print(dummies)
    df=pd.concat([df,dummies],axis=1)

     N
0    0
1    1
2    0
3    1
4    1
..  ..
317  1
318  0
319  0
320  0
321  0

[322 rows x 1 columns]
     W
0    0
1    1
2    1
3    0
4    0
..  ..
317  0
318  0
319  1
320  0
321  1

[322 rows x 1 columns]
     N
0    0
1    1
2    0
3    1
4    1
..  ..
317  1
318  0
319  0
320  0
321  0

[322 rows x 1 columns]


In [8]:
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,...,League,Division,PutOuts,Assists,Errors,Salary,NewLeague,N,W,N
0,293,66,1,30,29,14,1,293,66,1,...,A,E,446,33,20,535.925882,A,0,0,0
1,315,81,7,24,38,39,14,3449,835,69,...,N,W,632,43,10,475.000000,N,1,1,1
2,479,130,18,66,72,76,3,1624,457,63,...,A,W,880,82,14,480.000000,A,0,1,0
3,496,141,20,65,78,37,11,5628,1575,225,...,N,E,200,11,3,500.000000,N,1,0,1
4,321,87,10,39,42,30,2,396,101,12,...,N,E,805,40,4,91.500000,N,1,0,1


In [9]:
df=df.drop(df[categorical_columns],axis=1)
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,N,W,N
0,293,66,1,30,29,14,1,293,66,1,30,29,14,446,33,20,535.925882,0,0,0
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,632,43,10,475.000000,1,1,1
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,880,82,14,480.000000,0,1,0
3,496,141,20,65,78,37,11,5628,1575,225,828,838,354,200,11,3,500.000000,1,0,1
4,321,87,10,39,42,30,2,396,101,12,48,46,33,805,40,4,91.500000,1,0,1


In [ ]:
def outliers(df,column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df[column] = df[column].apply(lambda x: min(max(x, lower_bound), upper_bound))


In [ ]:
X=df.drop(df['Salary'],axis=1)
y=df['Salary']

In [ ]:
for cols in X:
    outliers(X,cols)

In [ ]:
scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)

In [ ]:
X_train,x_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.3,random_state=42)